## Extracting Annuli From Real Data

In the above examples we used model data to demonstrate how to measure the rotation velocity, but we didn't discuss how to get this data from your observations. We need a `spectra` array which has shape `(N, M)` where `N` is the number of spectra and `M` is the length of the velocity axis. We also need `theta`, which is a 1D, `N` length array of the polar angles of the specta. Finally we need `velax`, a length `M` 1D array of the velocity values. There are multiple ways to achieve this, however you can get the necessary values using the functions in the `rotationmap` class using a 3D cube rather than the 2D cubes it was designed to take. 

### Using `rotationmap` to Extract Spectra

First, constrain the geometry of the source in order to define an annulus. This include the source inclination and position angle on the sky. For more spatially resolved sources this may also include a flared emission surface. A quick way to do this is with `rotationmap.fit_keplerian` in `fit_cube`. See the "Fitting Rotation Maps" notebook for more information on how to do this. **Note:** For this you need to use a 2D moment map rather than the full 3D cube we use in the following steps.

Secondly, use these parmameters, (`x0`, `y0`, `inc`, `PA`), to mask a region within some bounding radii. This again can be easily achieved with the `rotationmap` class and the `disk_coords` function after initializing the class with the 3D cube from which you wish to extract spectra:

```python
    # Load the cube and find the pixel coordinates.
    from eddy.fit_cube import rotationmap
    cube = rotationmap('path/to/cube.fits')
    rvals, tvals, _ = cube.disk_coords(x0=x0, y0=y0, inc=inc, PA=PA)
```

which will return the deprojected radial values, `rvals` and polar angles, `tvals`, for each pixel. As discussed in the "Fitting Rotation Maps" notebook, we can also consider a flared emission surface described with the `z0` and `psi` parameters which `disk_coords` also accepts, along with a `tilt` value which describe the rotation of the disk.

A simple 2D mask can then be made with Numpy,

```python
    # Generate the mask.
    mask = np.logical_and(rvals >= r_min, rvals <= r_max)
    mask = mask.flatten()
```

where `r_min` and `r_max` are the inner and outer radii of the annulus in arcseconds. and which can be applied to the `cube.data` and `tvals` arrays, such as:

```python
    # Apply the mask to the data.
    spectra = cube.data.reshape(cube.data.shape[0], -1)
    spectra = spectra.flatten()[:, mask].T
    theta = tvals.flatten()[mask]
```

For the velocity axis, this isn't something that `rotationmap` does automatically as it works with just 2D images (moment maps). However you can extract it using something like:

```python
    # Get the velocity axis (assuming it is the third axis).
    velax = np.arange(cube.header['naxis3']) - cube.header['crpix3'] + 1.0
    velax = cube.header['crval3'] + velax * cube.header['cdelt3']
```

Remember to make sure this is velocity, as sometimes the spectral axis is given in frequency. This again is a simple coversion,

```python
    # Convert frequency to velocity.
    nu = cube.header['restfreq']
    velax = 2.9979e8 * (nu - velax) / nu
```

These can then be used to generate an `annulus` instance.